In [1]:
import os
import sys
import time
sys.path.append("/home/ziniuw/zero-shot-cost-estimation")
from models.zero_shot_models.specific_models.model import zero_shot_models
from cross_db_benchmark.benchmark_tools.database import DatabaseSystem
from models.training.train import train_default, train_readout_hyperparams

Using backend: pytorch


In [2]:
from copy import copy

import numpy as np
import optuna
import torch
import torch.optim as opt
from tqdm import tqdm

from cross_db_benchmark.benchmark_tools.utils import load_json
from models.dataset.dataset_creation import create_dataloader
from models.training.checkpoint import save_checkpoint, load_checkpoint, save_csv
from models.training.metrics import MAPE, RMSE, QError
from models.training.utils import batch_to, flatten_dict, find_early_stopping_metric
from models.zero_shot_models.specific_models.model import zero_shot_models

def training_model_loader(workload_runs,
                test_workload_runs,
                statistics_file,
                target_dir,
                filename_model,
                optimizer_class_name='Adam',
                optimizer_kwargs=None,
                final_mlp_kwargs=None,
                node_type_kwargs=None,
                model_kwargs=None,
                tree_layer_name='GATConv',
                tree_layer_kwargs=None,
                hidden_dim=32,
                batch_size=32,
                output_dim=1,
                epochs=0,
                device='cpu',
                plan_featurization_name=None,
                max_epoch_tuples=100000,
                param_dict=None,
                num_workers=1,
                early_stopping_patience=20,
                trial=None,
                database=None,
                limit_queries=None,
                limit_queries_affected_wl=None,
                skip_train=False,
                seed=0):
    if model_kwargs is None:
        model_kwargs = dict()

    # seed for reproducibility
    torch.manual_seed(seed)
    np.random.seed(seed)

    target_test_csv_paths = []
    if test_workload_runs is not None:
        for p in test_workload_runs:
            test_workload = os.path.basename(p).replace('.json', '')
            target_test_csv_paths.append(os.path.join(target_dir, f'test_{filename_model}_{test_workload}.csv'))

    # create a dataset
    loss_class_name = final_mlp_kwargs['loss_class_name']
    label_norm, feature_statistics, train_loader, val_loader, test_loaders = \
        create_dataloader(workload_runs, test_workload_runs, statistics_file, plan_featurization_name, database,
                          val_ratio=0.15, batch_size=batch_size, shuffle=True, num_workers=num_workers,
                          pin_memory=False, limit_queries=limit_queries,
                          limit_queries_affected_wl=limit_queries_affected_wl, loss_class_name=loss_class_name)

    if loss_class_name == 'QLoss':
        metrics = [RMSE(), MAPE(), QError(percentile=50, early_stopping_metric=True), QError(percentile=95),
                   QError(percentile=100)]
    elif loss_class_name == 'MSELoss':
        metrics = [RMSE(early_stopping_metric=True), MAPE(), QError(percentile=50), QError(percentile=95),
                   QError(percentile=100)]

    # create zero shot model dependent on database
    model = zero_shot_models[database](device=device, hidden_dim=hidden_dim, final_mlp_kwargs=final_mlp_kwargs,
                                       node_type_kwargs=node_type_kwargs, output_dim=output_dim,
                                       feature_statistics=feature_statistics, tree_layer_name=tree_layer_name,
                                       tree_layer_kwargs=tree_layer_kwargs,
                                       plan_featurization_name=plan_featurization_name,
                                       label_norm=label_norm,
                                       **model_kwargs)
    # move to gpu
    model = model.to(model.device)
    optimizer = opt.__dict__[optimizer_class_name](model.parameters(), **optimizer_kwargs)
    csv_stats, epochs_wo_improvement, epoch, model, optimizer, metrics, finished = \
        load_checkpoint(model, target_dir, filename_model, optimizer=optimizer, metrics=metrics, filetype='.pt')
    return test_loaders, model

def load_model(workload_runs,
            test_workload_runs,
               statistics_file,
               target_dir,
               filename_model,
               hyperparameter_path,
              device='cpu',
              max_epoch_tuples=100000,
              num_workers=1,
              loss_class_name='QLoss',
              database=None,
              seed=0,
              limit_queries=None,
              limit_queries_affected_wl=None,
              max_no_epochs=None,
              skip_train=False
              ):
    """
    Reads out hyperparameters and trains model
    """
    print(f"Reading hyperparameters from {hyperparameter_path}")
    hyperparams = load_json(hyperparameter_path, namespace=False)

    p_dropout = hyperparams.pop('p_dropout')
    # general fc out
    fc_out_kwargs = dict(p_dropout=p_dropout,
                         activation_class_name='LeakyReLU',
                         activation_class_kwargs={},
                         norm_class_name='Identity',
                         norm_class_kwargs={},
                         residual=hyperparams.pop('residual'),
                         dropout=hyperparams.pop('dropout'),
                         activation=True,
                         inplace=True)
    final_mlp_kwargs = dict(width_factor=hyperparams.pop('final_width_factor'),
                            n_layers=hyperparams.pop('final_layers'),
                            loss_class_name=loss_class_name,
                            loss_class_kwargs=dict())
    tree_layer_kwargs = dict(width_factor=hyperparams.pop('tree_layer_width_factor'),
                             n_layers=hyperparams.pop('message_passing_layers'))
    node_type_kwargs = dict(width_factor=hyperparams.pop('node_type_width_factor'),
                            n_layers=hyperparams.pop('node_layers'),
                            one_hot_embeddings=True,
                            max_emb_dim=hyperparams.pop('max_emb_dim'),
                            drop_whole_embeddings=False)
    final_mlp_kwargs.update(**fc_out_kwargs)
    tree_layer_kwargs.update(**fc_out_kwargs)
    node_type_kwargs.update(**fc_out_kwargs)

    train_kwargs = dict(optimizer_class_name='AdamW',
                        optimizer_kwargs=dict(
                            lr=hyperparams.pop('lr'),
                        ),
                        final_mlp_kwargs=final_mlp_kwargs,
                        node_type_kwargs=node_type_kwargs,
                        tree_layer_kwargs=tree_layer_kwargs,
                        tree_layer_name=hyperparams.pop('tree_layer_name'),
                        plan_featurization_name=hyperparams.pop('plan_featurization_name'),
                        hidden_dim=hyperparams.pop('hidden_dim'),
                        output_dim=1,
                        epochs=200 if max_no_epochs is None else max_no_epochs,
                        early_stopping_patience=20,
                        max_epoch_tuples=max_epoch_tuples,
                        batch_size=hyperparams.pop('batch_size'),
                        device=device,
                        num_workers=num_workers,
                        seed=seed,
                        limit_queries=limit_queries,
                        limit_queries_affected_wl=limit_queries_affected_wl,
                        skip_train=skip_train
                        )

    assert len(hyperparams) == 0, f"Not all hyperparams were used (not used: {hyperparams.keys()}). Hence generation " \
                                  f"and reading does not seem to fit"

    param_dict = flatten_dict(train_kwargs)
    
    test_loaders, model = training_model_loader(workload_runs, test_workload_runs, statistics_file, target_dir, filename_model, 
                          param_dict=param_dict, database=database, **train_kwargs)
    
    return test_loaders, model

def validate_model(val_loader, model, epoch=0, epoch_stats=None, metrics=None, max_epoch_tuples=None,
                   custom_batch_to=batch_to, verbose=False, log_all_queries=False):
    model.eval()

    with torch.autograd.no_grad():
        val_loss = torch.Tensor([0])
        labels = []
        preds = []
        probs = []
        sample_idxs = []

        # evaluate test set using model
        test_start_t = time.perf_counter()
        val_num_tuples = 0
        for batch_idx, batch in enumerate(tqdm(val_loader)):
            if max_epoch_tuples is not None and batch_idx * val_loader.batch_size > max_epoch_tuples:
                break

            val_num_tuples += val_loader.batch_size

            input_model, label, sample_idxs_batch = custom_batch_to(batch, model.device, model.label_norm)
            sample_idxs += sample_idxs_batch
            output = model(input_model)

            # sum up mean batch losses
            val_loss += model.loss_fxn(output, label).cpu()

            # inverse transform the predictions and labels
            curr_pred = output.cpu().numpy()
            curr_label = label.cpu().numpy()
            if model.label_norm is not None:
                curr_pred = model.label_norm.inverse_transform(curr_pred)
                curr_label = model.label_norm.inverse_transform(curr_label.reshape(-1, 1))
                curr_label = curr_label.reshape(-1)
           
            preds.append(curr_pred.reshape(-1))
            labels.append(curr_label.reshape(-1))

        if epoch_stats is not None:
            epoch_stats.update(val_time=time.perf_counter() - test_start_t)
            epoch_stats.update(val_num_tuples=val_num_tuples)
            val_loss = (val_loss.cpu() / len(val_loader)).item()
            print(f'val_loss epoch {epoch}: {val_loss}')
            epoch_stats.update(val_loss=val_loss)

        labels = np.concatenate(labels, axis=0)
        preds = np.concatenate(preds, axis=0)
        return labels, preds

In [3]:
database = DatabaseSystem.POSTGRES
workload_runs = ["../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s4_c8220.json",
                 "../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s5_c8220.json", 
                 "../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s6_c8220.json"]
test_workload_runs = ["../zero-shot-data/runs/parsed_plans/imdb_full/job_full_c8220.json"]
statistics_file = "../zero-shot-data/runs/parsed_plans/statistics_workload_combined.json"
target_dir = "../zero-shot-data/evaluation/job_full_tune/"
filename_model = "imdb_full_0"
hyperparameter_path = "/home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_best_config.json"
limit_queries = 1000
ft_filename_model = filename_model + f"_ft_{limit_queries}"

test_loaders, model = load_model(workload_runs, test_workload_runs, statistics_file, target_dir, filename_model, 
                  hyperparameter_path, database=database)

Reading hyperparameters from /home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_best_config.json
No of Plans: 50722
No of Plans: 77
Successfully loaded checkpoint from epoch 85 (85 csv rows) in 0.064 secs


In [4]:
true, pred = validate_model(test_loaders[0], model)
qerror = np.maximum(true/pred, pred/true)
print("performance without finetuning:")
for i in [50, 90, 95, 99, 100]:
    print(np.percentile(qerror, i))

100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.06s/it]

performance without finetuning:
1.2624925374984741
2.1067908763885503
2.556484699249268
6.628351116180374
13.535557746887207


In [7]:
model = train_readout_hyperparams(workload_runs, test_workload_runs, statistics_file, target_dir, ft_filename_model,
                          hyperparameter_path, num_workers=16, database=database, max_no_epochs=50, 
                          limit_queries=limit_queries, limit_queries_affected_wl=3
)

Reading hyperparameters from /home/ziniuw/zero-shot-cost-estimation/setup/tuned_hyperparameters/tune_est_best_config.json
Capping workload ../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s4_c8220.json after 333 queries
Stopping now
Capping workload ../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s5_c8220.json after 333 queries
Stopping now
Capping workload ../zero-shot-data/runs/parsed_plans/imdb_full/complex_workload_400k_s6_c8220.json after 333 queries
Stopping now
No of Plans: 1005
No of Plans: 77
PostgresZeroShotModel(
  (loss_fxn): QLoss()
  (fcout): Sequential(
    (0): FcLayer(
      (layers): Sequential(
        (0): Linear(in_features=128, out_features=192, bias=True)
        (1): LeakyReLU(negative_slope=0.01, inplace=True)
      )
    )
    (1): FcLayer(
      (layers): Sequential(
        (0): Linear(in_features=192, out_features=192, bias=True)
        (1): LeakyReLU(negative_slope=0.01, inplace=True)
      )
    )
    (2): FcLayer(


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.03it/s]


val_loss epoch 5: 0.8347772359848022
val_mse: 10.6745 [best: 6.5581]
val_mape: 2.1939 [best: 2.8307]
val_median_q_error_50: 2.1982 [best: 2.2543]
New best model for val_median_q_error_50
val_median_q_error_95: 10.6802 [best: 11.3514]
val_median_q_error_100: 18.4472 [best: 21.8811]
epochs_wo_improvement: 0
Saved checkpoint to ../zero-shot-data/evaluation/job_full_tune/imdb_full_0_pg_est_ft_1000.pt in 0.065 secs
Epoch 6


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.02it/s]


val_loss epoch 6: 0.7558313608169556
val_mse: 17.7687 [best: 6.5581]
val_mape: 1.8009 [best: 2.1939]
val_median_q_error_50: 2.1602 [best: 2.1982]
New best model for val_median_q_error_50
val_median_q_error_95: 10.5310 [best: 10.6802]
val_median_q_error_100: 16.3334 [best: 18.4472]
epochs_wo_improvement: 0
Saved checkpoint to ../zero-shot-data/evaluation/job_full_tune/imdb_full_0_pg_est_ft_1000.pt in 0.061 secs
Epoch 7


...

val_loss epoch 48: 0.29744502902030945
val_mse: 3.9903 [best: 3.9646]
val_mape: 0.3769 [best: 0.3197]
val_median_q_error_50: 1.3362 [best: 1.2728]
val_median_q_error_95: 2.1857 [best: 2.3248]
val_median_q_error_100: 4.2493 [best: 4.4016]
epochs_wo_improvement: 4
Saved checkpoint to ../zero-shot-data/evaluation/job_full_tune/imdb_full_0_pg_est_ft_1000.pt in 0.063 secs
Epoch 49


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.05it/s]


val_loss epoch 49: 0.31218430399894714
val_mse: 4.3808 [best: 3.9646]
val_mape: 0.4157 [best: 0.3197]
val_median_q_error_50: 1.3403 [best: 1.2728]
val_median_q_error_95: 2.2187 [best: 2.1857]
val_median_q_error_100: 4.3901 [best: 4.2493]
epochs_wo_improvement: 5
Saved checkpoint to ../zero-shot-data/evaluation/job_full_tune/imdb_full_0_pg_est_ft_1000.pt in 0.074 secs
Early stopping kicked in due to no improvement in 20 epochs
Starting validation for ../zero-shot-data/evaluation/job_full_tune/test_imdb_full_0_pg_est_ft_1000_job_full_c8220.csv
Reloading best model


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/it]

val_loss epoch 50: 0.5266938209533691
val_mse: 6.2942 [best: 3.9646]
val_mape: 0.6153 [best: 0.3197]
val_median_q_error_50: 1.4630 [best: 1.2728]
val_median_q_error_95: 7.1657 [best: 2.1857]
val_median_q_error_100: 11.6235 [best: 4.2493]


In [8]:
true, pred = validate_model(test_loaders[0], model)
qerror = np.maximum(true/pred, pred/true)
print(f"performance with {limit_queries} finetuning:")
for i in [50, 90, 95, 99, 100]:
    print(np.percentile(qerror, i))

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.15it/s]

performance with 1000 finetuning:
1.4629887342453003
3.233961677551271
7.165688800811769
10.138860626220692
11.623549461364746
